In [1]:
import sqlite3
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import numpy as np
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

import pickle

In [2]:
con = sqlite3.connect(r"C:\Users\reekithak\Downloads\drugs.db")

In [3]:
df = pd.read_sql("SELECT * from train_set0",con)

In [5]:
df

,cid,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,...,emb23,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,target
0,3081361,4.719912,-1.287492,5.796409,3.760270,-2.509900,-3.106354,6.347282,3.284466,-5.520523,...,-8.252186,-0.474333,5.553354,-3.007781,-6.632124,-2.022637,-5.149702,-5.259633,-5.783206,"1D4H,1FLT,1Y6A,1EVT,1IVO,1Y6A,1FLT,1JU5,2XB7,2..."
1,5327236,1.248603,-0.253252,-0.254269,0.804871,-0.980929,-0.430769,0.525774,-0.733890,-0.735165,...,-0.211978,-0.483960,-1.107684,0.974848,-0.160632,2.125608,0.882338,-1.016395,1.250610,"1D4H,1D4K"
2,5327235,1.409250,-0.664244,-0.052777,1.251166,-0.900441,-0.416737,0.396938,-1.056493,-0.664980,...,-0.250652,-0.127260,-1.264243,1.083983,-0.324660,2.211021,0.841747,-0.969820,1.200209,"1D4H,1D4K"
3,5327234,0.862983,-0.649309,0.103232,0.856554,-0.948067,-0.355748,-0.125225,-0.140954,0.125626,...,0.018531,-0.510167,-0.576433,1.177396,-0.220560,0.872600,0.954063,-0.638187,0.492565,1D4H
4,3009319,1.577372,-0.625860,0.031868,0.781114,-1.194058,-0.452654,0.164674,-0.567973,-0.532195,...,-0.213216,-0.307816,-1.132520,0.917929,-0.634182,2.012652,1.027961,-0.934430,1.287832,"1D4H,1D4K"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509978,91898556,0.295068,0.197552,0.545401,-0.036647,-0.673342,0.732977,-0.100805,-0.109900,-0.150411,...,1.179746,-0.527813,0.696605,0.478272,-0.318172,0.227475,0.437032,-1.003407,0.291253,4P8Q
509979,91898557,0.282878,-0.028419,0.690323,0.104092,-0.818978,0.629611,-0.164922,-0.003292,-0.337344,...,1.254489,-0.622289,0.656799,0.441361,-0.309904,0.212362,0.625125,-1.108965,0.287895,4P8Q
509980,91898558,0.427225,0.084797,0.526330,-0.104751,-0.709305,0.643887,-0.037825,-0.040398,-0.266387,...,1.175247,-0.633522,0.592134,0.494120,-0.343090,0.163036,0.505035,-1.020813,0.345524,4P8Q
509981,91898559,0.434695,0.076641,0.547011,0.077848,-0.662812,0.583030,-0.142893,-0.155146,-0.241697,...,1.195542,-0.686928,0.568743,0.457031,-0.410817,0.212401,0.603722,-0.803557,0.082198,4P8Q


In [6]:
cids = df["cid"].unique()
unique_pbdid = set(",".join(df["target"].values.flatten().tolist()).split(","))
pbdid_dict = {v:k for k, v in dict(enumerate(unique_pbdid)).items()}

In [8]:
X = []
y = []
print("Preparing Data")
for cid in tqdm(cids, total=len(cids), leave=False):
    indices = np.where(df["cid"] == cid)[0]
    xi = df.iloc[indices, 1:-1].values
    X.append(xi)
    yi = np.zeros(len(pbdid_dict))
    targets = ",".join(df.iloc[indices, -1].values.tolist()).split(",")
    targets = [pbdid_dict[_id] for _id in targets if _id != ""]
    yi[targets] = 1
    y.append(yi)

X = np.array(X).reshape(-1, 32)
scaler = StandardScaler()
scaler.fit(X)
y = np.array(y)

np.save('X.npy',X)
np.save('y.npy',y)

  0%|                                                                             | 3/509983 [00:00<5:56:58, 23.81it/s]

Preparing Data


In [17]:
X.shape

(509983, 32)

In [2]:
X = np.load(r'C:\Users\reekithak\train0\X0.npy') ; y = np.load(r"C:\Users\reekithak\train0\y0.npy")

In [5]:
X.shape, y.shape

((509983, 32), (509983, 2237))

In [45]:
x_t, x_v, y_t, y_v = train_test_split(X, y, test_size=0.01, random_state=42)

In [7]:
import winsound 
  
freq = 100
dur = 50
  
# loop iterates 5 times i.e, 5 beeps will be produced. 
for i in range(0, 5):     
    winsound.Beep(freq, dur)     
    freq+= 100
    dur+= 50

In [8]:
X.shape , y.shape

((509983, 32), (509983, 2237))

In [18]:
x_t.shape , y_t.shape

((494683, 32), (494683, 2237))

In [20]:
y_t[0].shape

(2237,)

In [11]:
def batches(l, n):
    
    for i in range(0, len(l), n):
        if i + n >= len(l):
            yield l[i:len(l)]
        else:
            yield l[i:i+n]

In [6]:
from sklearn.linear_model import SGDClassifier
import random
import pickle
import time
import tqdm
from sklearn.naive_bayes import MultinomialNB
clf2 = MultinomialNB()

In [13]:
shuffledRange = list(range(len(x_t)))

In [14]:
len(shuffledRange)

494683

In [32]:
n_iter = 1

In [31]:
t1 = time.time()
for n in range(n_iter):
    random.shuffle(shuffledRange)
#     x_t = x_t[shuffledRange, :]
#     y_t = y_t[shuffledRange, :]
    print("ITER ",n)
    count = 0
    print("Init_Batches = ",len(shuffledX))
    for batch in batches(shuffledRange, 1024):
        x_batch = x_t[batch, :]
        y_batch = y_t[batch, :]
        clf2.partial_fit(x_batch, y_batch)
        print('Batch ',count)
        count+=1
pickle.dump(clf2,open("final_model.sav",'wb'))
print(time.time()-t1)

ITER  0
Init_Batches =  494683


ValueError: y should be a 1d array, got an array of shape (1024, 2237) instead.

In [31]:
#Simple Evaluation


In [3]:
import tensorflow as tf

In [35]:
def batch_gen(X, y, bs):
    indices = list(range(X.shape[0]))
    random.shuffle(indices)
    while True:
        for i in range(0, len(indices) - (len(indices) % bs), bs):
            if i + bs > len(indices):
                batch_x = X[i:, :]
                batch_y = y[i:, :]
            else:
                batch_x = X[i:i+bs, :]
                batch_y = y[i:i+bs, :]
            yield (batch_x, batch_y)

In [15]:
# for x_, y_ in batch_gen(X, y, bs=1024):
#     yhat = model.predict(x_)
#     print(yhat.shape, y_.shape)
#     break

(1024, 2237) (1024, 2237)


In [56]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(32,), activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(1024, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2048, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(y.shape[1], activation="sigmoid")
])

In [57]:
# logits_loss = tf.keras.losses.SparseCategoricalCrossentropy()
# model.compile(optimizer="adam", loss=logits_loss, metrics=["accuracy"])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [58]:
bs = 1024
EPOCHS = 50
steps_per_epoch = x_t.shape[0] // bs
val_steps = x_v.shape[0] // bs
train_gen = batch_gen(x_t, y_t, bs=bs)
val_gen = batch_gen(x_v, y_v, bs=bs)

In [59]:
model.fit_generator(gen, steps_per_epoch = steps_per_epoch, epochs=EPOCHS, verbose=1)

Epoch 1/50
493/493 [==============================] - 92s 188ms/step - loss: 0.0114 - accuracy: 0.0789s - loss: 0.0114 - accuracy: 0.07
Epoch 2/50
493/493 [==============================] - 93s 188ms/step - loss: 0.0027 - accuracy: 0.3676
Epoch 3/50
493/493 [==============================] - 92s 187ms/step - loss: 0.0017 - accuracy: 0.5384
Epoch 4/50
493/493 [==============================] - 93s 188ms/step - loss: 0.0013 - accuracy: 0.6282
Epoch 5/50
493/493 [==============================] - 93s 189ms/step - loss: 0.0011 - accuracy: 0.6675
Epoch 6/50
493/493 [==============================] - 93s 189ms/step - loss: 9.6528e-04 - accuracy: 0.6958
Epoch 7/50
493/493 [==============================] - 94s 191ms/step - loss: 8.8415e-04 - accuracy: 0.7086TA: 34s - loss: 9 - ETA: 29s -  - ETA: 22s - loss: 9.2424e-04 - accuracy: 0 - ETA: 21s - loss: 9.2157e-04 - ac - ETA: 18s - loss: 9.1394e-04  - ETA: 15s 
Epoch 8/50
493/493 [==============================] - 94s 190ms/step - loss: 8.2297e-

In [62]:
model.save("graph0-ep50.h5")